In [1]:
import gspread

gc = gspread.oauth()

In [2]:
wb_deepwork = gc.open('Deep Work')
wb_tracker = gc.open('Personal Tracking')
sh_focus = wb_deepwork.worksheet('Focus')
cells_focus = sh_focus.get_all_records()

In [3]:
def get_cells(gc, workbook_name, worksheet_name):
    workbook = gc.open(workbook_name)
    worksheet = workbook.worksheet(worksheet_name)
    cells = worksheet.get_all_records()

    return cells

In [4]:
cells_focus = get_cells(gc, workbook_name='Deep Work', worksheet_name='Focus')

In [5]:
import pandas as pd, numpy as np
from quant_helper.data_cleaning import cols_to_datetime, format_datetime_index

df_focus = (
    pd.DataFrame(cells_focus)
    .pipe(cols_to_datetime, cols=['Start', 'Stop'])
    .assign(duration = lambda x: (x['Stop'] - x['Start']).divide(pd.Timedelta(hours=1)))
    .dropna(axis=0, subset=['Type'])
)

In [6]:
df_focus.tail()

,Task,Start,Stop,Type,Quality,Notes,Phone Checks,Date,Hours,Checks/Hr,MA(10),,duration
162,Logging for Markets Science Scripts,2020-07-13 12:44:52,2020-07-13 15:50:00,Deep,7,Chat with BD,0,,,,,,3.085556
163,Logging for Markets Science Scripts,2020-07-13 16:30:00,2020-07-13 18:00:00,Shallow,7,,0,,,,,,1.500000
164,Clear out daily tasks,2020-07-14 07:37:11,2020-07-14 08:03:14,Shallow,9,,0,,,,,,0.434167
165,Troubleshoot python logging,2020-07-14 08:03:33,2020-07-14 08:22:44,Deep,10,,0,,,,,,0.319722
166,Work on Modern Renaissance,2020-07-14 08:22:44,2020-07-14 10:05:48,Deep,9,,1,,,,,,1.717778


In [7]:
import matplotlib.pyplot as plt

summary_focus_daily = (
    df_focus
    .groupby([df_focus['Start'].dt.date, 'Type'])
    ['duration']
    .sum()
    .unstack()
    .pipe(format_datetime_index)
    .resample('D')
    .last()
    .fillna(0)
)

In [8]:
summary_focus_daily

Type,Deep,Education,Meeting,None,Shallow
date,,,,,
2020-01-09,3.152222,0.000000,0.00,0.0,0.000000
2020-01-10,5.115278,0.000000,0.00,0.0,0.000000
2020-01-11,0.000000,0.000000,0.00,0.0,0.000000
2020-01-12,4.779722,0.000000,0.00,0.0,0.000000
2020-01-13,2.497500,0.000000,0.00,0.0,0.000000
...,...,...,...,...,...
2020-07-10,1.275833,0.000000,0.75,0.0,2.562500
2020-07-11,4.231667,0.000000,0.00,0.0,1.591944
2020-07-12,0.000000,0.000000,0.00,0.0,0.000000


In [9]:
# def convert_col_dtypes(df, cols, dtypes):
#     for col, dtype in zip(cols, dtypes):
#         df[col] = df[col].astype(dtype)

#     return df

from quant_helper.data_cleaning import convert_col_dtypes

In [10]:
summary_focus_daily_formatted = (
    summary_focus_daily
    .reset_index()
    .pipe(convert_col_dtypes, ['date'], [str])
)

In [11]:
summary_focus_daily_formatted

Type,date,Deep,Education,Meeting,None,Shallow
0,2020-01-09,3.152222,0.000000,0.00,0.0,0.000000
1,2020-01-10,5.115278,0.000000,0.00,0.0,0.000000
2,2020-01-11,0.000000,0.000000,0.00,0.0,0.000000
3,2020-01-12,4.779722,0.000000,0.00,0.0,0.000000
4,2020-01-13,2.497500,0.000000,0.00,0.0,0.000000
...,...,...,...,...,...,...
183,2020-07-10,1.275833,0.000000,0.75,0.0,2.562500
184,2020-07-11,4.231667,0.000000,0.00,0.0,1.591944
185,2020-07-12,0.000000,0.000000,0.00,0.0,0.000000
186,2020-07-13,4.933333,2.424722,0.00,0.0,1.988611


In [12]:
def update_gsheet_pandas(worksheet, df):
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [13]:
sh_work = wb_tracker.worksheet('Work')

In [14]:
update_gsheet_pandas(sh_work, summary_focus_daily_formatted)

# Logging Stuff

In [62]:
from discord_webhook import DiscordWebhook, DiscordEmbed